In [2]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import confusion_matrix
from datasets import Dataset
import torch

c:\Users\oskio\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Read the CSV file using pandas
df = pd.read_csv('training_dataset.csv', encoding='utf-8')
dataset = Dataset.from_pandas(df)

# Define training and testing split
dataset = dataset.train_test_split(test_size=0.2)

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [5]:
# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Prepare the dataset for PyTorch
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

Map: 100%|██████████| 83362/83362 [00:27<00:00, 3009.99 examples/s]


In [6]:
# Intilize the model and move it to the GPU
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6)
model.to(torch.device('cuda'))

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',
    optim="adamw_torch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,
)

# Initialize the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
)

# Train the model
trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  1%|          | 500/62523 [04:08<8:37:26,  2.00it/s]

{'loss': 0.7709, 'grad_norm': 11.126877784729004, 'learning_rate': 1.9841018505190092e-05, 'epoch': 0.02}


  2%|▏         | 1000/62523 [08:27<9:24:49,  1.82it/s]

{'loss': 0.2293, 'grad_norm': 5.901440620422363, 'learning_rate': 1.9681077363530224e-05, 'epoch': 0.05}


  2%|▏         | 1500/62523 [12:40<8:12:17,  2.07it/s] 

...

100%|█████████▉| 62500/62523 [9:07:06<00:11,  1.97it/s]

{'loss': 0.0817, 'grad_norm': 1.2306817770004272, 'learning_rate': 1.5034467316027702e-08, 'epoch': 3.0}


                                                       
100%|██████████| 62523/62523 [9:21:01<00:00,  1.86it/s]

{'eval_loss': 0.09125309437513351, 'eval_runtime': 820.9237, 'eval_samples_per_second': 101.547, 'eval_steps_per_second': 6.348, 'epoch': 3.0}
{'train_runtime': 33661.2577, 'train_samples_per_second': 29.718, 'train_steps_per_second': 1.857, 'train_loss': 0.09934960999019109, 'epoch': 3.0}


TrainOutput(global_step=62523, training_loss=0.09934960999019109, metrics={'train_runtime': 33661.2577, 'train_samples_per_second': 29.718, 'train_steps_per_second': 1.857, 'total_flos': 2.632102289241477e+17, 'train_loss': 0.09934960999019109, 'epoch': 3.0})

In [9]:
predicted = trainer.predict(tokenized_datasets['test']).predictions.argmax(-1)
expected = tokenized_datasets['test']['labels']
cm = confusion_matrix(predicted, expected)
print(cm)
print(f'Accuracy: {cm.trace() / cm.sum()}')

100%|██████████| 5211/5211 [13:35<00:00,  6.39it/s]

[[23677    16     1   470   122     0]
 [   12 28074  1948    16     6   259]
 [    0    69  4964     0     0     0]
 [  180    17     0 10865   520     0]
 [  328    14     0    38  8275     1]
 [   23     4     1     0   787  2675]]
Accuracy: 0.9420359396367649


In [10]:
# Save the model
model.save_pretrained('sentiment_analysis/model/saved_model')
tokenizer.save_pretrained('sentiment_analysis/model/saved_tokenizer')

('sentiment_analysis/model/saved_tokenizer\\tokenizer_config.json',
 'sentiment_analysis/model/saved_tokenizer\\special_tokens_map.json',
 'sentiment_analysis/model/saved_tokenizer\\vocab.txt',
 'sentiment_analysis/model/saved_tokenizer\\added_tokens.json')